In [1]:
import tensorflow as tf

In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [3]:
pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/949.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/949.2 kB ? eta -:--:--
     ------------------- ---------------- 524.3/949.2 kB 621.2 kB/s eta 0:00:01
     ----------------------------- ------ 786.4/949.2 kB 745.8 kB/s eta 0:00:01
     ------------------------------------ 949.2/949.2 kB 771.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pypr

In [4]:
import yfinance as yf

In [6]:
data = yf.download('AAPL', start='2015-01-01', end='2021-12-31')

C:\Users\shaq6\AppData\Local\Temp\ipykernel_19260\587856783.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL', start='2015-01-01', end='2021-12-31')
[*********************100%***********************]  1 of 1 completed


In [7]:
data = data[['Close']]

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [9]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [10]:
time_step = 60
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)


In [11]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [12]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

c:\Users\shaq6\.anaconda\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [14]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - loss: 0.0035
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 5.1238e-04
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - loss: 4.9788e-04
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 4.5530e-04
Epoch 5/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 4.1211e-04
Epoch 6/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 4.3150e-04
Epoch 7/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 3.9886e-04
Epoch 8/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 3.3749e-04
Epoch 9/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 154ms/step - loss: 3.2579e-04
Epoch 10/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 10s 118ms/step - loss: 3.2340e-04


In [15]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step


In [16]:
print(predicted_stock_price[:5])

[[117.34552 ]
 [118.796394]
 [120.28258 ]
 [121.81646 ]
 [123.20509 ]]
